# Imports

In [8]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
import bigram
from utils import emotion_scores
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model Setup

In [3]:
corpus_file_path = '../data/corpus.txt'
corpus = open(corpus_file_path, 'r').read()
print(len(corpus))

238101


In [4]:
print(emotion_scores("lmao"))

[{'label': 'sadness', 'score': 0.04048539698123932}, {'label': 'joy', 'score': 0.2761611044406891}, {'label': 'love', 'score': 0.006484143901616335}, {'label': 'anger', 'score': 0.5209840536117554}, {'label': 'fear', 'score': 0.14821957051753998}, {'label': 'surprise', 'score': 0.007665674667805433}]


In [118]:
model=bigram.BigramLM(corpus)
model.set_token()
model.calculate_bigrams()
laplace_matrix = model.get_laplace_matrix()
kn_matrix = model.set_kn_matrix(d = 0.75)

Model Initialized 🟢
Tokens Set 🟢


Populating Bigram Matrix...: 100%|██████████| 46531/46531 [00:00<00:00, 931220.34it/s]


All Matrices Calculated 🟢


Calculating Kneser-Ney Matrix...: 100%|██████████| 5429/5429 [00:29<00:00, 186.06it/s]


# Generating Emotion for all non-zero Bigrams

In [89]:
tokens = model.get_token()
count_matrix = model.get_count_matrix()

In [90]:
emotions = {}
for i, token in tqdm(enumerate(tokens)):
    for j, token2 in enumerate(tokens):
        if count_matrix[i][j] > 0:
            emotions[(token, token2)] = emotion_scores(str(token + " " + token2))

with open('emotions.pkl', 'wb') as f:
    pickle.dump(emotions, f)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

# Generating 50 Sentences each of every emotion


In [119]:
emotions=["sadness","joy","love","anger","fear","surprise"]
for i in emotions:
    #used count matrix use some other matrix like laplace or kneser-nay
    model.generate_sentences(model.get_kn_matrix(),emotion=i,word_limit=15,no_of_sentences=50,alpha = 0.6,beta = 0.4)

Generating sadness Matix: 5429it [00:08, 621.34it/s]
Generating Sentence: 100%|██████████| 50/50 [00:01<00:00, 49.28it/s]


Sentences Generated + Stored 🟢


Generating joy Matix: 5429it [00:09, 553.45it/s]
Generating Sentence: 100%|██████████| 50/50 [00:00<00:00, 71.35it/s]


Sentences Generated + Stored 🟢


Generating love Matix: 5429it [00:08, 621.03it/s]
Generating Sentence: 100%|██████████| 50/50 [00:00<00:00, 73.35it/s]


...

Sentences Generated + Stored 🟢


Generating fear Matix: 5429it [00:08, 626.69it/s]
Generating Sentence: 100%|██████████| 50/50 [00:00<00:00, 67.85it/s]


Sentences Generated + Stored 🟢


Generating surprise Matix: 5429it [00:08, 644.28it/s]
Generating Sentence: 100%|██████████| 50/50 [00:00<00:00, 71.79it/s]


Sentences Generated + Stored 🟢


# Training SVC Classifier


In [21]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [94]:
corpus_file = "..\data\corpus.txt"
labels_file = "..\data\labels.txt"
with open(corpus_file) as f:
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(f)
    fts = vectorizer.get_feature_names_out()
print(X)

with open(labels_file) as l:
    y = l.read().splitlines()

  (0, 3060)	0.39399399834967097
  (0, 2282)	0.2605195037319864
  (0, 2279)	0.26908509588959934
  (0, 2756)	0.37424850064917514
  (0, 1009)	0.3493721155355762
  (0, 3589)	0.29587146656272845
  (0, 792)	0.29923130828043504
  (0, 1493)	0.31561696426322433
  (0, 1730)	0.06957240243423639
  (0, 2195)	0.25323754913162366
  (0, 4468)	0.3109755120185638
  (1, 3413)	0.24483193083519053
  (1, 4352)	0.10654075239969912
  (1, 2931)	0.10600712072565037
  (1, 2866)	0.1852146252445593
  (1, 2732)	0.15107607914585053
  (1, 4356)	0.24005070277480042
  (1, 3194)	0.2896883899813202
  (1, 3115)	0.09092845250228027
  (1, 8)	0.11339300079539945
  (1, 5355)	0.2568796633395555
  (1, 5238)	0.14399272611188504
  (1, 3798)	0.22001308723193067
  (1, 401)	0.12896817474176356
  (1, 3319)	0.1471918187925502
  :	:
  (2397, 1239)	0.4413532714679378
  (2397, 2522)	0.24789733492565907
  (2397, 401)	0.32631694934040917
  (2397, 1730)	0.12943018232788128
  (2398, 3473)	0.5067785294902316
  (2398, 4634)	0.3077091570750903


In [78]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [95]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10], 'gamma':[0.1, 0.01, ], "degree": [2,3,4] }
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X, y)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'degree': [2, 3, 4],
                         'gamma': [0.1, 0.01], 'kernel': ('linear', 'rbf')})

In [115]:

print(clf.best_params_)
print(clf.best_score_)

{'C': 10, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
0.7474999999999999


In [120]:
emotions=["sadness","joy","love","anger","fear","surprise"]
dataset = {}
for emotion in emotions:
    path = ".\emotion_text\gen_"+emotion+".txt"
    with open(path, 'r') as file:
        dataset[emotion] = file.read().splitlines()
            
        

In [121]:
def emotion_accuracy(emotion, dataset, clf):
    X = vectorizer.transform(dataset[emotion])
    y = [emotion]*len(dataset[emotion])
    return clf.score(X, y)

for emotion in emotions:
    print(emotion, emotion_accuracy(emotion, dataset, clf)) 
   
print("Average accuracy: ", np.mean([emotion_accuracy(emotion, dataset, clf) for emotion in emotions]))

sadness 0.68
joy 0.48
love 0.9
anger 0.36
fear 0.5
surprise 0.76
Average accuracy:  0.6133333333333333
